# Hierarchical Indexing

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [3]:
data

a  1    0.333090
   2   -0.544515
   3    1.778346
b  1   -1.282702
   3    0.846028
c  1   -1.200037
   2   -0.769135
d  2    1.138523
   3    0.403418
dtype: float64

In [4]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

### Slicing

In [5]:
data['b']

1   -1.282702
3    0.846028
dtype: float64

In [6]:
data['b':'c']

b  1   -1.282702
   3    0.846028
c  1   -1.200037
   2   -0.769135
dtype: float64

In [7]:
data.loc[['b', 'd']]

b  1   -1.282702
   3    0.846028
d  2    1.138523
   3    0.403418
dtype: float64

In [8]:
#inner selection
data.loc[:, 2]

a   -0.544515
c   -0.769135
d    1.138523
dtype: float64

### stack()/unstack()

In [9]:
#hierarchical => decompressed
#Outer index = index labels
#inner index = column lables
data.unstack()

,1,2,3
a,0.333090,-0.544515,1.778346
b,-1.282702,NaN,0.846028
c,-1.200037,-0.769135,NaN
d,NaN,1.138523,0.403418


In [10]:
#stack => compressing
data.unstack().stack()

a  1    0.333090
   2   -0.544515
   3    1.778346
b  1   -1.282702
   3    0.846028
c  1   -1.200037
   2   -0.769135
d  2    1.138523
   3    0.403418
dtype: float64

### DataFrame

In [11]:
frame = pd.DataFrame(np.arange(4 * 3).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

In [12]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

### Naming hierarchical axis labels

In [13]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']

In [14]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [15]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

## 1. Reordering and Sorting Levels

### 1.1 swaplevel()

In [16]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

### 1.2 sort_index()
* Only a single level can be sorted, default is outer

In [17]:
#key2 here is sorted by index values
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [18]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## 2. Summary Statistics by Level

In [19]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [20]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## 3. Indexing with a DataFrame’s columns

In [21]:
frame = pd.DataFrame({'a': np.arange(7),
                      'b': np.arange(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})

In [22]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


### 3.1 set_index()
* Selecting some columns as index labels in the DF

In [23]:
frame_2 = frame.set_index(['c', 'd'])
frame_2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [24]:
#leaving the indexed columns in the columns
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

### 3.2 reset_index()
* Opposite of set_index()

In [25]:
frame_2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
